<a href="https://colab.research.google.com/github/SkaiLord/ML-Lab/blob/main/ML_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# cleaning, drop rows, replace null values: with pad and bfill, nearest neighbours;
# normalisation -> age -> drop values >2.2 outliers
# Box plot, q3,q1,IQR -> find outliers and remove outliers
# Feature encoding

1. Cleaning the Dataset
Drop Rows: We'll drop any rows with missing essential information that can't be filled meaningfully.
Replace Null Values: For the columns with missing data, we'll replace null values using the following methods:
pad: Forward fill (use the previous value to fill the missing one).
bfill: Backward fill (use the next value to fill the missing one).
Nearest neighbors: Use the values of the nearest neighbors (e.g., through K-Nearest Neighbors imputation).
2. Normalization
Age: Normalize the Age column and remove any outliers where the values are greater than 2.2 (based on a z-score or other criteria).
3. Outlier Detection and Removal
Box Plot, Q1, Q3, IQR: Calculate Q1, Q3, and the Interquartile Range (IQR) for the numeric columns. Identify outliers and remove them.
4. Feature Encoding
Encode categorical variables like Sex and Embarked using techniques such as one-hot encoding or label encoding.

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/titanic.csv')
# show no.of false and true
df['Age'].isna().value_counts()

,count
Age,
False,714
True,177


In [ ]:
# Drop rows where 'Embarked' is missing as it's essential
df.dropna(subset=['Embarked'], inplace=True)
# Fill missing 'Age' values using padding (forward fill)
df['Age'].ffill(inplace=True)
df
# Fill remaining missing 'Age' values using backfill
df['Age'].bfill(inplace=True)
df

# Normalize 'Age' column
scaler = MinMaxScaler()
df['Age'] = scaler.fit_transform(df['Age'].values.reshape(-1, 1))

# Remove outliers in 'Age' column
df = df[df['Age'] <= 2.2]

In [ ]:
# Fill missing 'Age' values with forward fill
df['Age'].fillna(method='pad', inplace=True)
# Fill remaining missing 'Age' values using backfill
df['Age'].fillna(method='bfill', inplace=True)

# Fill missing 'Cabin' values using nearest neighbors (if applicable), fill with the most frequent value in 'Cabin'
df['Cabin'].fillna(df['Cabin'].mode()[0], inplace=True)

<ipython-input-37-6292938ef22b>:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Age'].fillna(method='pad', inplace=True)
<ipython-input-37-6292938ef22b>:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Age'].fillna(method='bfill', inplace=True)


In [ ]:
from scipy import stats

# Calculate z-scores
df['Age_zscore'] = stats.zscore(df['Age'])

# Filter out outliers
df = df[df['Age_zscore'] <= 2.2]

# Drop the z-score column as it's no longer needed
df.drop(columns=['Age_zscore'], inplace=True)

<ipython-input-38-8776a2566b37>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['Age_zscore'], inplace=True)


In [ ]:
# Calculate Q1 (25th percentile) and Q3 (75th percentile) for 'Fare'
Q1 = df['Fare'].quantile(0.25)
Q3 = df['Fare'].quantile(0.75)
IQR = Q3 - Q1

# Define bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Remove outliers from 'Fare'
df = df[(df['Fare'] >= lower_bound) & (df['Fare'] <= upper_bound)]


7.8958 26.0 18.1042


In [ ]:
# Encode 'Sex' using label encoding (0 for female, 1 for male)
df['Sex'] = df['Sex'].map({'male': 1, 'female': 0})

# One-hot encode 'Embarked'
df = pd.get_dummies(df, columns=['Embarked'], drop_first=True)


In [ ]:
# Save the cleaned dataset
df.to_csv('titanic_cleaned.csv', index=False)
df = pd.read_csv('titanic_cleaned.csv')
# show no.of false and true
df['Age'].isna().value_counts()

,count
Age,
False,734
